## This notebook is for downscaling the Landsat ET data from 30m to 10m resolution. Subsequently the downscaled ImageCollection is exported to the Project Asset folder in Google Earth Engine.
---

In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
import ee
import geemap

ee.Initialize(project="thurgau-irrigation")

In [3]:
from vegetation_period_NDVI.data_loading import load_sentinel2_data, add_time_data
from vegetation_period_NDVI.time_series import (
    extract_time_ranges,
    get_harmonic_ts,
    add_time_data,
)
from utils.composites import harmonized_ts
from utils.date_utils import print_collection_dates, create_centered_date_ranges
from utils.harmonic_regressor import HarmonicRegressor
from utils.ee_utils import back_to_float, back_to_int, export_image_to_asset, aggregate_to_monthly


from typing import List

### Constants

In [4]:
YEAR = 2022
BUFFER_DAYS = 15
TIME_STEPS = 12
TIME_STEP_TYPE = "monthly"

### Step 1: Define the AOI

In [5]:
cantonal_borders_asset = (
    "projects/thurgau-irrigation/assets/Zuerich/Zuerich_bound"
)

aoi_feature_collection = ee.FeatureCollection(cantonal_borders_asset)
aoi_geometry = aoi_feature_collection.geometry()
aoi_geometry = aoi_geometry.simplify(500)
aoi_buffered = aoi_geometry.buffer(100)

### Step 2: Load the gap filled Landsat ET product at 30m resolution
#### I mistakenly exported the asset with 10m resolution, but the pixels are still 30m. So, I need to adjust the metadata to reflect the correct resolution.

In [6]:
def update_projection_scale(image):
    """Update the image projection to 30m scale while preserving values."""
    return image.setDefaultProjection(image.projection().atScale(30))

# Update all images in the collection to have correct 30m scale
landsat_ET_30m = (ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Zuerich/Landsat_ET_gap_filled_monthly_30m_ZH_2018-2022"
    )
    .map(lambda img: back_to_float(img, 100))  # First convert values back to float
    .map(update_projection_scale)  # Then update the projection scale
)


### Step 3: Get the sentinel 2 data, and compute the vegetation indices

In [7]:
s2collection = load_sentinel2_data(year=YEAR, aoi=aoi_buffered)

In [8]:
landsat_ET_30m_list = landsat_ET_30m.toList(landsat_ET_30m.size())

time_intervals = create_centered_date_ranges(landsat_ET_30m_list, buffer_days=BUFFER_DAYS)

bands = ["B3", "B4", "B8", "B11", "B12"]

options = {"agg_type": "mosaic", "mosaic_type": "least_cloudy", "band_name": "NDVI"}

s2_harmonized = harmonized_ts(
    masked_collection=s2collection,
    band_list=bands,
    time_intervals=time_intervals,
    options=options,
)


In [9]:
def compute_vegetation_indexes(image: ee.Image) -> ee.Image:
    """
    Compute vegetation indexes for a given image

    Args:
        image (ee.Image): The image to compute the vegetation indexes for

    Returns:
        ee.Image: The input image with the vegetation indexes

    """
    ndvi = image.normalizedDifference(["B8", "B4"]).rename("NDVI")
    ndwi = image.normalizedDifference(["B3", "B8"]).rename("NDWI")
    ndbi = image.normalizedDifference(["B11", "B8"]).rename("NDBI")
    return image.addBands(ndvi).addBands(ndwi).addBands(ndbi)

s2_harmonized_w_vegetation_indexes = s2_harmonized.map(compute_vegetation_indexes)

### Step 4: Fill gaps in the harmonized time series using harmonic regression

In [10]:
indexes = ["NDVI", "NDWI", "NDBI"]

s2_harmonized_w_vegetation_indexes = s2_harmonized_w_vegetation_indexes.map(
    add_time_data
)

s2_harmonized_gaps_filled = s2_harmonized_w_vegetation_indexes

for index in indexes:
    regressor = HarmonicRegressor(
        omega=1.5, max_harmonic_order=2, vegetation_index=index
    )

    regressor.fit(s2_harmonized_w_vegetation_indexes)
    fitted_collection = regressor.predict(s2_harmonized_w_vegetation_indexes)

    fitted_collection = fitted_collection.map(
        lambda img: img.select(["fitted"]).rename(f"fitted_{index}")
    )

    s2_harmonized_gaps_filled = s2_harmonized_gaps_filled.map(
        lambda img: img.addBands(
            fitted_collection.filterDate(img.date()).first().select([f"fitted_{index}"])
        )
    )

### Step 5: Downscale the Landsat ET data to 10m resolution

In [11]:
from utils.downscale_anything_10m import Downscaler, resample_collection

In [12]:
def process_and_export_downscaled_ET(
    downscaler: Downscaler,
    s2_indices: ee.ImageCollection,
    independent_vars: ee.ImageCollection,
    dependent_vars: ee.ImageCollection,
    aoi: ee.Geometry,
    year: str,
    scale_coarse: float,
    scale_fine: float = 10,
    time_steps: int = 36,
    time_step_type: str = "dekadal",
) -> List[ee.batch.Task]:
    """
    Process and export downscaled WaPOR ET images to Earth Engine assets.

    Args:
        downscaler (Downscaler): The Downscaler object used to downscale the images.
        s2_indices (ee.ImageCollection): The Sentinel-2 indices image collection.
        independent_vars (ee.ImageCollection): The resampled independent variables image collection.
        dependent_vars (ee.ImageCollection): The dependent variables image collection.
        aoi (ee.Geometry): The area of interest geometry.
        year (str): The year for which the images are processed.
        scale_coarse (float): The scale of the images before downscaling.
        scale_fine (float): The scale of the images after downscaling.
        time_steps (int): Number of time steps in the year (36 for dekadal, 12 for monthly).
        time_step_type (str): Type of time step ("dekadal" or "monthly").

    Returns:
        List[ee.batch.Task]: A list of export tasks for the downscaled images.
    """
    s2_indices_list = s2_indices.toList(s2_indices.size())
    independent_vars_list = independent_vars.toList(independent_vars.size())
    dependent_vars_list = dependent_vars.toList(dependent_vars.size())

    tasks = []
    for i in range(time_steps):
        if time_step_type == "dekadal":
            j = i % 3 + 1
            m = i // 3 + 1
            date = ee.Date.fromYMD(int(year), m, j * 10 - 9)
            time_step_name = f"{m:02d}_D{j}"
        elif time_step_type == "monthly":
            m = i + 1
            date = ee.Date.fromYMD(int(year), m, 1)
            time_step_name = f"{m:02d}"
        else:
            raise ValueError("time_step_type must be either 'dekadal' or 'monthly'")

        s2_index = ee.Image(s2_indices_list.get(i))
        ind_vars = ee.Image(independent_vars_list.get(i))
        dep_vars = ee.Image(dependent_vars_list.get(i))

        # Perform downscaling
        et_image_downscaled = downscaler.downscale(
            coarse_independent_vars=ind_vars,
            coarse_dependent_var=dep_vars,
            fine_independent_vars=s2_index,
            geometry=aoi,
            resolution=scale_coarse,
        )

        # Post-process the downscaled image
        et_image_downscaled = back_to_int(et_image_downscaled, 100)

        task_name = f"Downscaled_Landsat_ET_gap_filled_{time_step_type}_10m_ZH_{year}_{time_step_name}"
        asset_id = f"projects/thurgau-irrigation/assets/Zuerich/Landsat_ET_gap_filled_{time_step_type}_10m_ZH_{year}/{task_name}"

        task = export_image_to_asset(
            et_image_downscaled,
            asset_id,
            task_name,
            year,
            aoi,
            crs="EPSG:32632",
            scale=scale_fine,
        )
        tasks.append(task)

    return tasks

In [13]:
independent_bands = ["fitted_NDVI", "fitted_NDBI", "fitted_NDWI"]
dependent_band = ["fitted_ET"]

s2_indices = s2_harmonized_gaps_filled.select(independent_bands)
independent_vars = resample_collection(s2_indices, landsat_ET_30m)
dependent_vars = landsat_ET_30m.select(dependent_band)

scale = landsat_ET_30m.first().projection().nominalScale().getInfo()


# Initialize the Downscaler
downscaler = Downscaler(
    independent_vars=independent_bands, dependent_var=dependent_band[0]
)

tasks = process_and_export_downscaled_ET(
    downscaler,
    s2_indices,
    independent_vars,
    dependent_vars,
    aoi_buffered,
    YEAR,
    scale_coarse=scale,
    scale_fine=10,
    time_steps=TIME_STEPS,
    time_step_type=TIME_STEP_TYPE,
)

# You can add additional code here to monitor the tasks if needed
print(f"Started {len(tasks)} export tasks.")

Exporting Downscaled_Landsat_ET_gap_filled_monthly_10m_ZH_2022_01 for 2022 to projects/thurgau-irrigation/assets/Zuerich/Landsat_ET_gap_filled_monthly_10m_ZH_2022/Downscaled_Landsat_ET_gap_filled_monthly_10m_ZH_2022_01
Using projection EPSG:32632 at 10m resolution
Exporting Downscaled_Landsat_ET_gap_filled_monthly_10m_ZH_2022_02 for 2022 to projects/thurgau-irrigation/assets/Zuerich/Landsat_ET_gap_filled_monthly_10m_ZH_2022/Downscaled_Landsat_ET_gap_filled_monthly_10m_ZH_2022_02
Using projection EPSG:32632 at 10m resolution
Exporting Downscaled_Landsat_ET_gap_filled_monthly_10m_ZH_2022_03 for 2022 to projects/thurgau-irrigation/assets/Zuerich/Landsat_ET_gap_filled_monthly_10m_ZH_2022/Downscaled_Landsat_ET_gap_filled_monthly_10m_ZH_2022_03
Using projection EPSG:32632 at 10m resolution
Exporting Downscaled_Landsat_ET_gap_filled_monthly_10m_ZH_2022_04 for 2022 to projects/thurgau-irrigation/assets/Zuerich/Landsat_ET_gap_filled_monthly_10m_ZH_2022/Downscaled_Landsat_ET_gap_filled_monthly_1

### Sanity check

In [14]:
# downscaled_landsat = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/Zuerich/Landsat_ET_gap_filled_monthly_10m_ZH_2022"
# ).map(lambda img: back_to_float(img, 100))

In [15]:
# Map = geemap.Map()

# image_downscaled = ee.Image(downscaled_landsat.toList(downscaled_landsat.size()).get(5))
# image_corrected = ee.Image(landsat_ET_30m.toList(landsat_ET_30m.size()).get(5))

# vis_params = {
#     "bands": ["downscaled"],
#     "min": 0,
#     "max": 200,
#     "palette": ["#000000", "#0000FF", "#00FF00", "#FFFF00", "#FF0000"],
# }

# vis_params2 = {
#     "bands": ["fitted_ET"],
#     "min": 0,
#     "max": 200,
#     "palette": ["#000000", "#0000FF", "#00FF00", "#FFFF00", "#FF0000"],
# }

# Map.addLayer(image_downscaled, vis_params, "Downscaled ET")
# Map.addLayer(image_corrected, vis_params2, "Corrected ET")

# Map.centerObject(aoi_buffered, 10)
# Map